E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [1]:
import pandas as pd

# Read the provided N1.csv file
nfa_description_N1 = pd.read_csv("N1.csv")
nfa_description_N1


ModuleNotFoundError: No module named 'pandas'